# Lecture 14: Toeplitz matrices + matrix functions

## Mid-term test statistics
<img src='test-stat.png' >

## Syllabus
**Week 1:** Matrices, vectors, matrix/vector norms, scalar products & unitary matrices  
**Week 2:** TAs-week (Strassen, FFT, a bit of SVD)  
**Week 3:** Matrix ranks, singular value decomposition, linear systems, eigenvalues  
**Week 4:** Matrix decompositions: QR, LU, SVD + test + structured matrices start  
**Week 5:** Iterative methods, preconditioners, matrix functions

## Recap of the previous lecture
- Concept of Incomplete LU/Incomplete Cholesky (ILU(k), ILUT, ILU2)
- A brief note about algebraic multigrid (no details, because we need PDEs for the general multigrid concept) 
- Iterative methods for other NLA problems (SVD, eigenvalue problems)
- Other than sparse: Toeplitz matrices, FFT, circulant matrices.

## Today lecture

Today we have (again!) a very ambitious plan:

- Direct methods for Toeplitz matrices, Gohberg-Semencul formula
- Iterative methods for Toeplitz matrices
- Circulant preconditioners
- Two-dimensional (BTTB) matrices
- Matrix functions: basic concepts

## Reminder
First, we will remind the last part of the Tuesday lecture.

## Toeplitz matrices/circulant: one slide from last lecture

A matrix is called **Toeplitz** if its elements are defined as

$$a_{ij} = t_{i - j}.$$

A Toeplitz matrix is completely defined by its first column and first row (i.e., $2n-1$ parameters).

It can also be multiplied by vector fast using circulant embedding and FFT.

## Low displacement rank structure

The Toeplitz matrices belong to a more general class of **low displacement rank** matrices.

Define the **scaled periodic shift matrix** 

$Z_e$ that takes the vector $x$ and transforms it to a vector

$$Z_e x = \begin{bmatrix}
e x_{n-1} \\
x_1 \\
x_2 \\
\vdots \\
x_{n-2}
\end{bmatrix}
$$

What is the matrix form of this linear operator?

## Shift matrices, displacement matrices and Toeplitz matrices

Given a Toeplitz matrix $T$, we select any $e, f$ such that $ef \ne 1$ and define the displacement operator as

$$L(T) = Z_e T - T Z_f.$$

For a Toeplitz matrix, $L(T)$ has **rank 2** (it has only first row and last column non-zero)

In [ ]:
import numpy as np
import scipy.linalg
n = 5 
c = np.zeros(n)
c[0] = -2
c[1] = 1
T = sp.linalg.toeplitz(c, c)
e = 0.5
f = 0.5
def Z_shift(e):
    return  np.diag(np.ones(n-1), -1)  + e * np.diag(np.ones(1), n-1)

Z1 = Z_shift(e)
Z2 = Z_shift(f)

print Z1.dot(T) - T.dot(Z2)

What about the inverse?

In [ ]:
import numpy as np
import scipy.linalg
n = 5 
c = np.zeros(n)
c[0] = -2
c[1] = 1
T = sp.linalg.toeplitz(c, c)
e = 0.5
f = 0.5
def Z_shift(e):
    return  np.diag(np.ones(n-1), -1)  + e * np.diag(np.ones(1), n-1)

Z1 = Z_shift(e)
Z2 = Z_shift(f)

Tinv = np.linalg.inv(T)

p1 = Z1.dot(Tinv) - Tinv.dot(Z2)
np.linalg.svd(p1)[1]

## Small displacement rank: definition

The matrix is said to be of **displacement rank $r$** with respect to the pair of generators 

$Z_e, Z_f$, if

$$L(T) = Z_e T - T Z_f = GH^{\top},$$

where $G$ is $n \times r$ and $H$ is $n \times r$.

It is similar to "discrete derivative"

## Theorem on the inverse structure

Let $T$ satisfy

$$Z_e T - T Z_f = GH ^{\top},$$

and let it be invertible.

Then we have

$$T^{-1} (Z_e T - T Z_f) T^{-1} = T^{-1} Z_e - Z_f T^{-1} = T^{-1} G H^{\top} T^{-1},$$

i.e. the inverse has **small displacement rank** with the reversed pair of generators $Z_e, Z_f$ (why?).

## Recovering a matrix from the displacement representation

Does the low-rank representation after the displacement operator describe the structure?

I.e. we need to solve the equation of the form


$$Z_e T - T Z_f = GH^{\top} = B$$

For a given right-hand side.

This is a linear system of equations in disguise! (Do you see that this is a linear system? What is the size of this linear system?)

## Sylvester equation
The equation above is a special case of the **Sylvester matrix equation** which has the general form

$$AX - X B = C,$$

with $A$, $B$ and $C$ given.

In general, this is a linear system with $\mathcal{O}(n^2)$ unknowns, and the computational cost is expected to be $n^6$.

Can we do better?

## Solving Sylvester equation by the reduction to triangular form

The Sylvester equation, in general, is solved with the help of the matrix decompositions.

Reduce the matrix $B$ into the triangular form:

$$B = U T U^*.$$

Then we have

$$A X - X U T U^* =C, \quad A Y - Y T = C U, \quad Y = X U,$$

and solving the equation with triangular matrix $B$ is **easy** (let us check it on whiteboard).

## Back to the particular case

For the particular case, we have
$$Z_e T - T Z_f = GH^{\top} = B,$$

and the solution is given by

$$ (e - f) T = \sum_{j = 1}^r Z_e(g_j) Z_f( J h_j), $$

where $Z_e$ is the **e-scaled circulant** generated by the vector, and $g_j$, and $h_j$ are the columns of the matrices $G$ and $H$,

and $J$ is the **per-identity matrix** (which has ones on the anti-diagonal).

For details, see [the paper by Victor Pan et. al.](http://ac.els-cdn.com/S0024379501003366/1-s2.0-S0024379501003366-main.pdf?_tid=c532e56a-680e-11e5-81a5-00000aab0f01&acdnat=1443685015_d1c190dd112773e7479235b435da3b7e)

## Gohberg-Semencul formula

Based on this idea and for the special case when $e = 0, f \rightarrow \inf$, we get the following famous formula for the inverse of the Toeplitz matrices as a sum 
of two products of **triangular** Toeplitz matrices:

Let $A$ be a Toeplitz matrix, and

$$A \begin{bmatrix} x_0 \\ x_1 \\ \vdots \\ x_n \end{bmatrix}=\begin{bmatrix} 1 \\ 0 \\ \vdots \\ 0 \end{bmatrix},
\quad 
A \begin{bmatrix} y_0 \\ y_1 \\ \vdots \\ y_n \end{bmatrix}=\begin{bmatrix} 0 \\ 0 \\ \vdots \\ 1 \end{bmatrix}
$$

then 

$$A^{-1} = \frac{1}{x_0} \begin{bmatrix} x_0 & 0 & \ldots & 0 \\ x_1 & x_0 & 0 & \ldots \\ \ldots & \ldots \\ x_n & \ldots & \ldots & x_0 \end{bmatrix}\begin{bmatrix} u_0 & u_1 & \ldots & 0 \\ 0 & u_0 & u_1 & \ldots \\ \ldots & \ldots \\ 0 & \ldots & \ldots & u_0 \end{bmatrix}-\frac{1}{x_0} \begin{bmatrix} 0 & 0 & \ldots & 0 \\ y_0 & 0 & 0 & \ldots \\ y_1 & y_0 & \ldots  \\ \ldots & \ldots & \\ y_{n-1} & \ldots & y_0 & 0 \end{bmatrix}\begin{bmatrix} 0 & v_0 & \ldots & 0 \\ 0 & 0 & v_0 & v_1 \\ \ldots & \ldots   \\ \ldots & \ldots & \ldots & v_0 \\ 0 & \ldots & \ldots & 0\end{bmatrix},$$

where $u_y = y_{n-i}, \quad v_i = x_{n-i}$.

The main meaning: the inverse can be recovered from **the first column** and the **last column** of the matrix.

For details, I refer to the book by Tyrtyshnikov "Brief introduction to numerical analysis". 

## Fast and superfast direct methods

For many years, these formulas gave rise to the fast $\mathcal{O}(n^2)$ and superfast $\mathcal{O}(n \log n)$ methods for Toeplitz matrices.

The basic idea is that you use **augmentation method**. 

Consider that you have computed the **inverse** of a $(n-1) \times (n-1)$ block of the Toeplitz matrix.

You only need to store two vectors to represent the inverse.

Then, the bigger matrix can be written in the block form.

$$T_n = \begin{bmatrix} T_{n-1} & a \\ b^{\top} & c \end{bmatrix}.$$

We only need to recompute the last and the first column!


## Recomputing the last and the first columns

Let us split the vector as 

$$x = \begin{bmatrix} x_1 & x_2 \end{bmatrix}.$$

Then,


$$T_{n-1} x_1 + a x_2 = e_1, \quad b^{\top} x_1 + c x_2 = 0.$$

Or, 

$$  x_1 = T^{-1}_{n-1} e_1 - T^{-1}_n a x_2.$$

Application of $T^{-1}_{n-1}$ costs $\mathcal{O}(n \log n)$ operations, 

thus $x_2$ will be recovered in the same number of operations as well. The total complexity is then $\mathcal{O}(n^2 \log n)$ operations.

**Superfast algorithms** are obtained in terms of reducing the problem to the computations with polynomials and using **block** elimination  

(in the Fourier style).

## Problems with fast and superfast methods

- Main problem: superfast can be very unstable, since submatrices can be singular during the process.
- Practical problems: No good open-source software (and in general with Toeplitz matrices, which is extremely bad).

## Iterative methods

Since direct methods can be really difficult to implement, people turned to **iterative methods**.

A natural idea is to use **circulants** as preconditioners, since they are easy to invert.

The first preconditioner was the preconditioner by **Raymond Chan and Gilbert Strang**, who proposed to take the first column of the matrix  

and use it to generate the circulant.

The second preconditioner is the **Tony Chan** preconditioner, which is also very natural: 

$$C = \arg \min_P \Vert P - T \Vert_F.$$

A simple formula for the entries of $C$ can be derived.

In [ ]:
import numpy as np
import scipy.linalg
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
n = 100
c = np.zeros(n)
c[0] = -2
c[1] = 1
Tm = sp.linalg.toeplitz(c, c)


c1 = sp.linalg.circulant(c) #Strang preconditioner
Fmat = 1.0/np.sqrt(n) * np.fft.fft(np.eye(n)) #Poor man's Fourier matrix

d2 = np.diag(Fmat.conj().dot(Tm).dot(Fmat))
c2 = Fmat.dot(np.diag(d2)).dot(Fmat.conj().T)


mat = np.linalg.inv(c1).dot(Tm)
ev = np.linalg.eigvals(mat).real
plt.plot(np.sort(ev), np.ones(n), 'o')
plt.xlabel('Eigenvalues for Strang preconditioner')
plt.gca().get_yaxis().set_visible(False)

mat = np.linalg.inv(c2).dot(Tm)
ev = np.linalg.eigvals(mat).real
plt.figure()
plt.plot(np.sort(ev), np.ones(n), 'o')
plt.xlabel('Eigenvalues for T. Chan Preconditioner')
plt.gca().get_yaxis().set_visible(False)


## Better circulant preconditioner

In fact, there is a much better one, and it is based on another idea.

The matrix $C$ is a good preconditioner for a matrix $T$ if and only if

$$T = C + R + E, $$

where $R$ is **small rank**, $E$ is **small norm**.

You can use (in a non-trivial way) this formulation to get  **optimal circulant preconditioner.**

## Yet another way (and probably the best)

This part is based on the ideas described in [the paper](http://amath.colorado.edu/faculty/martinss/Pubs/2004_toeplitz.pdf)

Since circulant matrices are diagonalized by the Fourier transform, then what happens with a Toeplitz matrix after the FFT?

In [ ]:
import numpy as np
import scipy.linalg
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
n = 100
c = np.zeros(n)
c = [1.0/(i - 0.5) for i in range(n)]

Tm = sp.linalg.toeplitz(c, c)


c1 = sp.linalg.circulant(c) #Strang preconditioner
Fmat = 1.0/np.sqrt(n) * np.fft.fft(np.eye(n)) #Poor man's Fourier matrix

T_tranf = Fmat.conj().dot(Tm).dot(Fmat)


plt.contourf(T_tranf.real)




## Main observation
The off-diagonal part has small rank! 

In [ ]:
np.linalg.svd(T_tranf[n/2:, :n/2])[1]

## Exact formula for the FFT-transformed Toeplitz

The FFT-transformed Toeplitz matrices has the form

$$ \widehat{T} = F^* T F, \quad \widehat{T}(m, n) = (v(m) - v(n)) \phi(m - n), \quad m \ne n,$$

where 

$$
  v(n) = t(N + n) - t(n), \quad \phi(m-n) = \frac{1}{2 \sqrt{N} \sin(\pi(m- n)/N)}.
$$

## Statement
The matrices with big off-diagonal blocks that are of low-rank can efficient inverted. 

There are many algorithms, but for the 1D-case methods based on so-called **quasi-separable matrices** can be used to 

get $\mathcal{O}(N \log^{-1} \varepsilon)$-complexity algorithms.

## General scheme
- Compute quasi-separable ($\approx$ low-rank approximation of off-diagonal blocks in a hierarchical fashion) approximation of the Fourier transformed matrix $\widehat{T}$
- Apply efficient algorithms for the quasi-separable matrices to get a solution (the inverse of a quasi-separable matrix is a quasi-separable matrix).

More details again not here, but in the FastPDE course, since these types of matrices frequently arise in the solution of PDEs and integral equations.

## Summary 
-  The low-displacement rank is the right class for the Toeplitz matrices
- Fast algorithms use augmentation method to iteratively recompute the inverse
- The most efficient algorithm reduce to the block low-rank structure.
- No efficient software!!! (probably we will write it some day).

## Some random notes
The concept of **low displacement rank** is valid also for other classes of matrices.

- Vandermonde matrix, $x^p_i$

- Cauchy matrix, $\frac{1}{x_i - y_j}$.

Guess what are the **generators**?

## Block case

A very interesting case is when you go from one-dimensional signals (say, audio) to two-dimensional and multidimensional case (i.e., images).

Then it is natural to index the components of the vector by a pair of indices, $x_{i_1 i_2}$, and the elements of the matrix, 

acting on such "vector" by 4 indices, $A_{i_1 i_2 j_1 j_2}$, where $(i_1, i_2)$ enumerates the rows, while $(j_1, j_2)$ enumerates 

the columns of the matrix.

Generalizations to higher dimensions are obvious.

## Block-Toeplitz-with-Toeplitz-Blocks

In the two-dimensional case the shift invariance gives rise to **two-level** Toeplitz matrices:

$$A_{i_1 i_2 j_1 j_2} = t_{(i_1 - j_1)(i_2 - j_2)}.$$

They can be multiplied by vector fast by embedding into **two-level circulant matrices** (Block Circulant with Circulant Blocks, BCCB), which are diagonalized by two-dimensional FFT.

$$C = F^*_2 D F_2, $$

where $$F_2 = F \otimes F.$$

## Solvers for BTTB matrices


There are no direct $\mathcal{O}(N \log N)$ solvers for BTTB matrices.

The fastest direct solver is the block version of the Gohberg-Semencul formula, which avoid one levels and has

$\mathcal{O}(N^{3/2})$ complexity.

Typically, iterative methods are used. 

However, there are negative results on using **BCCB** preconditioners, 

[How to prove that preconditioner can not be superlinear](http://www.ams.org/mcom/2003-72-243/S0025-5718-03-01506-0/)

but in practice they work quite good.

There are **attempts** to do multigrid methods, but they are not very successful.

## Summary-2

- For 1D case there is a lot.
- For the Block case one has to use iterative methods.
- Research is ongoing

## Matrix functions
Now, a little bit about matrix functions.

## Outline of this part

- What is a matrix function
- Matrix exponential
- (Some) applications

Book to read: [Nick Higham, Functions of matrices](http://www.google.ru/books?hl=ru&lr=&id=2Wz_zVUEwPkC&oi=fnd&pg=PR3&dq=Higham+matrix+function&ots=pTt6fpLGRX&sig=DgUuX-SpBZGin8CFUo-4MYnOcHE&redir_esc=y#v=onepage&q=Higham%20matrix%20function&f=false)

## What is a matrix function?
A matrix function is a function of a matrix. A matrix function **is not** a function, applied to the elements!

I.e., $B = \sqrt{A}$ **does not mean** that $B_{ij} = \sqrt{A}_{ij}$.

In [ ]:
import numpy as np
import scipy.linalg as sla 

A = 2 * np.eye(2)
A = np.array([[4, 4], [4, 4]])
sla.funm(A, lambda x: np.sqrt(x) )

## The simplest matrix function: matrix polynomial

It is very easy to define a matrix polynomial as  
$$
 P(A) = \sum_{k=0}^n c_k A^k.
$$
**Side-note:** [Hamilton-Cayley theorem](https://en.wikipedia.org/wiki/Cayley%E2%80%93Hamilton_theorem) states that $F(A) = 0$ where $F(\lambda) = \det(A - \lambda I)$.

## Matrix polynomials as building blocks
We can define a function of the matrix by **Taylor series**:  
$$
   f(A) = \sum_{k=0}^{\infty} c_k A^k.
$$
The convergence is understood as the convergence in some **matrix norm**.  

Example of such series is the **Neumann series**  
$$
  (I - F)^{-1} = \sum_{k=0}^{\infty} F^k,
$$
which is well defined for $\rho(F) < 1$.

## Matrix exponential series
The most well-known matrix function is **matrix exponential**. In the scalar case,  
$$
   e^x = 1 + x + \frac{x^2}{2} + \frac{x^3}{6} + \ldots = \sum_{k=0}^{\infty} \frac{x^k}{k!},
$$
and it directly translates to the matrix case:  
$$
    e^A = \sum_{k=0}^{\infty} \frac{A^k}{k!},
$$
the series that always converges (why?).

## Why matrix exponential is important
A **lot of** practical problems are reduced to a system of linear ODEs of the form  
$$
   \frac{dy}{dt} = Ay, \quad y(0) = y_0.
$$
The formal solution is given by $y(t) = e^{At} y_0$, so if we know  

$e^{At}$ (or can compute matrix-by-vector product fast) there is a big gain over the  

time-stepping schemes.

## How to compute matrix functions?

See [C. Van Loan, C. Moler, Nineteen Dubious Ways to Compute the Exponential of a Matrix, Twenty-Five Years Later](http://www.cs.cornell.edu/cv/researchpdf/19ways+.pdf)

The simplest way is to diagonalize the matrix:  
$$
  A = S \Lambda S^{-1}.
$$
where the columns of $S$ are **eigenvectors** of the matrix $A$,  then  

$$
   F(A) = S F(\Lambda) S^{-1}.
$$
Problem: **diagonalization can be unstable!** (and not every matrix is diagonalizable)

Let us look how matrices are diagonalizable:

In [ ]:
import numpy as np
eps = 0.5
p = 4
a = np.eye(p)
for i in xrange(p-1):
    a[i, i+1] = 1
    
a[p-1, 2] = eps
a = np.array(a)
val, vec = np.linalg.eig(a)
#print a
print np.linalg.norm(a - vec.dot(np.diag(val)).dot(np.linalg.inv(vec)))
#print 'S * D * S^{-1}:' 
#print vec.dot(np.diag(val)).dot(np.linalg.inv(vec))
print a

Now we can compute a function for **perturbed Jordan block.**

In [ ]:
import numpy as np
eps = 1e-14
p = 5
a = np.eye(p)
for i in xrange(p-1):
    a[i, i+1] = 1
    
a[p-1, 0] = eps
a = np.array(a)
val, vec = np.linalg.eig(a)
print np.linalg.norm(a - vec.dot(np.diag(val)).dot(np.linalg.inv(vec)))

fun = lambda x: np.exp(x)

#Using diagonalization
fun_diag = vec.dot(np.diag(fun(val))).dot(np.linalg.inv(vec))


#Using Schur
import scipy
fun_m = scipy.linalg.expm(a)
np.linalg.norm(fun_m - fun_diag)

## How funm function works
The exponential of a matrix is a special function, so there are special methods for its computation.  

For a general function,  there is a beautiful **Schur-Parlett algorithm**, which is based on the **Schur theorem**

## Schur-Parlett algorithm

Given a matrix $A$ we want to compute $F(A)$, and we only can evaluate $F$ at **scalar points**.  
First, we reduce $A$ to the **triangular form** as  
$$
   A = U T U^*.
$$
Therefore,  $F(A)=U F(T) U^*$

We only need to compute the function of triangular matrices.

## Computing functions of triangular matrices
We know values on the diagonals:  
and $$F_{ii} = F(T_{ii}),$$
and also we know that
$$F T = T F,$$
and we get

$$f_{ij} = t_{ij} \frac{f_{ii} - f_{jj}}{t_{ii} - t_{jj}} + \sum_{k=i+1}^{j-1} \frac{f_{ik} t_{kj} - t_{ki}f_{kj}}{t_{ii} - t_{jj}}.$$

## Summary for this part
- Matrix functions are not elementwise functions
- For general functions, Schur-Parlett algorithm is the best.

Next week we will finish the matrix functions part (matrix exponential in more details, sign function, square root) and also describe the cases where they appear.

## Take home message

- There are fast direct methods for Toeplitz but few people use them since there is no software
- Typically, iterative methods are used
- Know the definition of a BTTB matrix
- A general understanding of what a matrix function is and how it can be computed.

## Next week
Next week is the last study week. We will finish the matrix functions part and go for the 


**advanced topics.**

They include: 

- Compressed sensing, L1-norm minimization, optimal design (A-optimality, D-optimality, maximum-volume principle in low-rank matrix factorization)
- Multidimensional case: multilinear algebra 

# Questions?

In [113]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()